In [1]:
# Setup
import os
import numpy as np # For general mathematical operations
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import torchvision
import matplotlib.pyplot as plt # For plotting the results
from torchsummary import summary # For model summaries
from torch.utils.tensorboard import SummaryWriter # For writing into tensorboard
import nrrd # For reading and manipulating nrrd files
from glob import glob
import torch.nn.functional as F
%load_ext autoreload
%autoreload 3

# Develop focal loss

Specifically solve the alpha issue

In [119]:
alpha = torch.Tensor(
[0.5, 1.0, 4.0, 1.0, 4.0, 4.0, 1.0, 1.0, 3.0, 3.0]
)  # TODO: focal loss weights per channels from the paper



gamma = 2.0
dims = [2, 10, 20, 30, 30]
weights = torch.ones(dims)
alpha_transformed = (weights.transpose(1,-1)*alpha).transpose(1,-1).view(-1)

targets = torch.rand(dims)
inputs = (torch.rand(dims) + targets) / 2

# Add error specifically to channel 0
inputs[0,...] = torch.rand(dims[1:])

orig_input_shape = inputs.shape
# flatten label and prediction tensors
inputs = inputs.view(-1)
targets = targets.view(-1)
# first compute binary cross-entropy

BCE = F.binary_cross_entropy(inputs, targets, weight=alpha_transformed, reduction="mean")

BCE_EXP = torch.exp(-BCE)
focal_loss = (1 - BCE_EXP) ** gamma * BCE


focal_loss


tensor(1.2577)

# Then the dice loss


In [92]:
def forward(inputs, targets):
    inputs = inputs.view(-1)
    targets = targets.view(-1)

    intersection = (inputs * targets).sum()
    dice = (2.0 * intersection) / (inputs.sum() + targets.sum())

    return dice

In [157]:
dims = [2,10, 30, 20, 20]

targets = torch.rand(dims)
inputs = (torch.rand(dims) + targets) / 2

dice = 0

for batch in range(dims[0]):
    for channel in range(dims[1]):
        inputs_ch = inputs[batch,channel,...].view(-1)
        targets_ch = targets[batch,channel,...].view(-1)

        intersection = (inputs_ch * targets_ch).sum()
        dice += (2.0 * intersection) / (inputs_ch + targets_ch).sum()

dice /= dims[1]
dice /= dims[0]
dice, forward(inputs, targets)

(tensor(0.5830), tensor(0.5830))

In [96]:
#targets = torch.rand(dims)
#inputs = (torch.rand(dims) + targets) / 2

inputs = inputs.view(-1)
targets = targets.view(-1)

intersection = inputs*targets
divisor = inputs + targets

(intersection / divisor).sum()

tensor(35.1463)

In [145]:
diceloss = 0
dims = (18,27,27)
targets = torch.rand(dims)
inputs = (torch.rand(dims) + targets) / 2

p = inputs
y = targets
diceloss += (2*(p*y)/(p+y)).sum()

diceloss /= dims[1]

diceloss, forward(inputs, targets=targets)

(tensor(230.9566), tensor(0.5849))

In [130]:
targets = torch.randn(2, 3).round(decimals=0)
inputs = torch.Tensor([2])


print(inputs)
print(targets)
torch.cross(targets, torch.Tensor([2]), dim=-1)

tensor([2.])
tensor([[-1., -1.,  0.],
        [ 1., -0.,  0.]])


tensor([[-2.,  2.,  0.],
        [-0., -2.,  2.]])